In [1]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Preproccesing and recostructing a dataset from DLS
Few lines intro

**Learning objectives:**
1. Construct and use CIL processors Centre of rotation and Resizer to preprocess the data
2. Define the projector
3. Apply different reconstruction techniques to a real dataset

In [33]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from ccpi.framework import ImageData, ImageGeometry
from ccpi.framework import AcquisitionGeometry, AcquisitionData

from ccpi.optimisation.algorithms import CGLS, SIRT
from ccpi.optimisation.functions import Norm2Sq, L1Norm
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity
from ccpi.framework import BlockDataContainer

from ccpi.processors import Resizer, CenterOfRotationFinder

from ccpi.io import NEXUSDataReader
from ccpi.astra.operators import AstraProjectorSimple , AstraProjector3DSimple
from ccpi.astra.processors import FBP

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import scipy
from utilities import islicer, link_islicer
from utilities import plotter2D


### Read in the dataset

About the data?

In [3]:
## Set up a reader object pointing to the Nexus data set. Revise path as needed.
# The data is already  corrected for by flat and dark field.
path = os.path.join(sys.prefix, 'share','ccpi','24737_fd_normalised.nxs')
myreader = NEXUSDataReader(nexus_file=path)
data_raw = myreader.load_data()

In [4]:
#Look at the data set
print(type(data_raw))
print(data_raw)
islicer(data_raw, direction=0)

<class 'ccpi.framework.framework.AcquisitionData'>
Number of dimensions: 3
Shape: (91, 135, 160)
Axis labels: {0: 'angle', 1: 'vertical', 2: 'horizontal'}



interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=90), Output()), _dom_cl…

IntSlider(value=0, continuous_update=False, description='x', max=90)

### Use Resizer() to pre-proccess the data

We want to remove the top row data with the dark pixel

Use the processor Resizer()


In [5]:
start_row = 1
end_row = data_raw.shape[1]
roi_crop = [-1,(start_row, end_row),-1]

In [6]:
resizer = Resizer(roi=roi_crop)
resizer.set_input(data_raw)
data_cropped = resizer.get_output()

In [7]:
#Look at the data set
print(type(data_cropped))
print(data_cropped)
islicer(data_cropped, direction=0)

<class 'ccpi.framework.framework.AcquisitionData'>
Number of dimensions: 3
Shape: (91, 134, 160)
Axis labels: {0: 'angle', 1: 'vertical', 2: 'horizontal'}



interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=90), Output()), _dom_cl…

IntSlider(value=0, continuous_update=False, description='x', max=90)

### Use CenterOfRotationFinder()

In [8]:
# Set up CenterOfRotationFinder object to center data.
# Set the output of the normaliser as the input and execute to determine center.
cor = CenterOfRotationFinder()
cor.set_input(data_cropped)
center_of_rotation = cor.get_output()
print( "centre of rotation at pixel: ", center_of_rotation)

centre of rotation at pixel:  86.25


<span style="color:red;font-size:larger">**Exercise 1:**</span> Use Resizer to crop the pojections and correct the Centre of rotation offset.

In [9]:
#get image width
num_pixels_y = data_cropped.shape[2]
cor_shift = center_of_rotation - num_pixels_y/2

In [10]:
start_column = int(round(2.*cor_shift))
end_column = data_cropped.shape[2]

roi_crop = [-1,-1,(start_column, end_column)]
resizer = Resizer(roi=roi_crop)
resizer.set_input(data_cropped)
data_centred = resizer.get_output()

In [11]:
#Look at the data set
print(type(data_centred))
print(data_centred)
islicer(data_centred, direction=0)

<class 'ccpi.framework.framework.AcquisitionData'>
Number of dimensions: 3
Shape: (91, 134, 148)
Axis labels: {0: 'angle', 1: 'vertical', 2: 'horizontal'}



interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=90), Output()), _dom_cl…

IntSlider(value=0, continuous_update=False, description='x', max=90)

### Set up the data ready for ASTRA

ASTRA expects the data in the order `['vertical','angle','horizontal']` so we need to permute the dataset

In [12]:
print(data_centred)

Number of dimensions: 3
Shape: (91, 134, 148)
Axis labels: {0: 'angle', 1: 'vertical', 2: 'horizontal'}



In [13]:
data = data_centred.subset(dimensions=['vertical','angle','horizontal'])

In [14]:
print(data)

Number of dimensions: 3
Shape: (134, 91, 148)
Axis labels: {0: 'vertical', 1: 'angle', 2: 'horizontal'}



ASTRA takes the angles as radians so let's convert the angles geometry data

In [15]:
#convert the angles to radians
if(data.geometry.angle_unit == 'degree'):
    data.geometry.angle_unit = 'radian'
    data.geometry.angles = data.geometry.angles * np.pi /180.

And finally convert from intensity to attenuation data

In [16]:
#Convert from intensity to attenuation data
#how can I make sure thy don't execute this more than once?
#data.fill(-np.log(data.as_array()))
data.log(out=data)
data *= -1
#Look at the data set
print(type(data))
print(data)
islicer(data, direction=1) #remember we've transposed the data so angles is now the 2nd axis

<class 'ccpi.framework.framework.AcquisitionData'>
Number of dimensions: 3
Shape: (134, 91, 148)
Axis labels: {0: 'vertical', 1: 'angle', 2: 'horizontal'}



interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=90), Output()), _dom_cl…

IntSlider(value=0, continuous_update=False, description='x', max=90)

### Define the Geometry

#### Acquistion geometry
In the 2D example we used:<br>
`ag = AcquisitionGeometry(geom_type='parallel', dimension='2D', angles=angles, pixel_num_h=number_pixels_x)`<br>

For 3D we need to change the dimension description to ` dimension='3D'`, and pass the number of vertical pixels as `pixel_num_v`<br>

However we've been using the acquistion geometry throughout this notebook so let's just clone the version we've already set up.

#### Image geometry
In the 2D example we used:<br>
`ig = ImageGeometry(voxel_num_x = num_voxels_xy, voxel_num_y = num_voxels_xy)`

For ad 3D reconstruction we also need to pass the number of voxels we want in the $z$-direction as `voxel_num_z`

If you create the image geometry from the acquisiton geometry this is set to `pixel_num_h` by default.

In [17]:
# Create Acquisition Geometry
ag = data.geometry.clone()

# Create Image Geometry
ig = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                   voxel_num_y=ag.pixel_num_h, 
                   voxel_num_z=ag.pixel_num_v)

## FBP Reconstruction

Reconstruct the data set using the FBP processor from ASTRA

`from ccpi.astra.processors import FBP`

We Run this in the same way as the processors introduced above.

In [18]:
#Initialise the processor
fbp = FBP(ig, ag, device='gpu')

In [19]:
#set the input
fbp.set_input(data)

In [20]:
#Run the procesor
FBP_output = fbp.get_output()

In [21]:
#plot the results
islicer(FBP_output, direction=0)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

IntSlider(value=0, continuous_update=False, description='x', max=133)

### Define the projector

In the 2D example we used the 2D projector from ASTRA<br>
`'AstraProjectorSimple(volume_geometry, sinogram_geometry, device)`

Use ASTRA's 3D projector, note this projector is GPU only<br>
`AstraProjector3DSimple(volume_geometry, sinogram_geometry)`

In [22]:
# Define the projector object
print ("Define projector")
Cop = AstraProjector3DSimple(ig, ag)

Define projector


### Run SIRT

In [23]:
#setup SIRT
x_init = ig.allocate(0)
sirt = SIRT(x_init=x_init, operator=Cop, data=data, update_objective_interval = 10)
sirt.max_iteration = 1000

SIRT setting up
SIRT configured


In [24]:
#run the algorithm
sirt.run(100, verbose = True)

     Iter   Max Iter     Time/Iter            Objective
                               [s]                     
        0       1000         0.000          1.01151e+06
       10       1000         0.053          1.01151e+06
       20       1000         0.052          1.88900e+04
       30       1000         0.053          6.70392e+03
       40       1000         0.052          3.32638e+03
       50       1000         0.053          2.01420e+03
       60       1000         0.053          1.38821e+03
       70       1000         0.052          1.03907e+03
       80       1000         0.053          8.20673e+02
       90       1000         0.053          6.72625e+02
      100       1000         0.052          5.66454e+02


In [25]:
#plot the results
SIRT_output = sirt.get_output()
islicer(SIRT_output, direction=0)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

IntSlider(value=0, continuous_update=False, description='x', max=133)

### Run Tikhonov CGLS

In [48]:
#define the operator
alpha = 15
L = Gradient(ig)
operator_block = BlockOperator( Cop, alpha * L, shape=(2,1))

In [49]:
#define the data b
data_block = BlockDataContainer(data, L.range_geometry().allocate(0))

In [50]:
#setup Tikonov
x_init = ig.allocate(0)
cgls_tikhonov = CGLS(x_init=x_init, operator=operator_block, data=data_block, update_objective_interval = 10)
cgls_tikhonov.max_iteration = 1000

CGLS setting up
CGLS configured


In [51]:
#run the algorithm
cgls_tikhonov.run(100, verbose = True)

     Iter   Max Iter     Time/Iter            Objective
                               [s]                     
        0       1000         0.000          1.01151e+06
       10       1000         0.119          1.25464e+05
       20       1000         0.116          2.91400e+03
       30       1000         0.113          2.82487e+03
       40       1000         0.114          2.81703e+03
       50       1000         0.114          2.81604e+03
       60       1000         0.114          2.81596e+03
       68       1000         0.114          2.81596e+03
Tolerance is reached: 1e-06


In [52]:
#plot the results
CGLS_tikhonov_output = cgls_tikhonov.get_output()

islicer(CGLS_tikhonov_output, direction=0)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

IntSlider(value=0, continuous_update=False, description='x', max=133)

### Summary

In [53]:
#compare the outputs of unregularise and regularised CGLS

slicer1=islicer(SIRT_output, direction=0,minmax=(0,0.006),title='SIRT')
slicer2=islicer(CGLS_tikhonov_output, direction=0,minmax=(0,0.006),title='CGLS')
slicer3=islicer(FBP_output, direction=0,minmax=(0,0.006),title='FBP')

link_islicer(slicer1,slicer2,slicer3)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=133), Output()), _dom_c…